<a href="https://colab.research.google.com/github/favorov/ASmethodsBenchmarking/blob/master/OT_codon_usage_2_gram_negative_clades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

#trying to import ot
#if it does not work, pip install -- if fails, we will get error
try:
  import ot
except ImportError:
  !pip install pot
  import ot

goocola=True
try:
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/OT+Codon usage colab')
except ImportError:
  goocola=False
#Are we in google clabs?? If somebody cares, the goocola boolean says it
#Whatever, the current folder is ready for read/write
#if we are in collab, it is My Drive/OT+Codon usage colab 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
#extract information about the frequency distribution
#def extract_freq(a, n, nc):
#        N   = a[n - 1] + a[n]
#        out = [0] * nc
#        for m in range(nc):
#            out[m] = a[n - 1 + m] / N
#        return(out)

#def extract_freq_new(a, b):
#        N   = a + b
#        j   = [a, b]
#        out = [0] * 2
#        for m in range(2):
#            out[m] = j[m] / N
#        return(out)

#counts is list. Python get lists by reference, so we modify them
def counts_to_freqs(counts):
    length=len(counts)
    csum=sum(counts)
    for i in range(length):
        counts[i]=counts[i]/csum
    return(counts)





In [110]:

#just a test for pd.read_excel
file_name = 'phen_small.xlsx'

df = pd.read_excel(io = file_name, header = [0, 1])
print(df.head(5))  # print first 5 rows of the dataframe

  Phenylalanine (Phe/F)            ...                    
                  # CDS Organelle  ...       TTT       TTC
0               3888949   genomic  ...  25537177  18665561
1               2710660   genomic  ...   3479845  24457252
2               2141118   genomic  ...  12017184  12262306
3               1387878   genomic  ...    966339  14576929
4               1462644   genomic  ...  13657458   4995245

[5 rows x 7 columns]


In [111]:
#extract test data

M = [[0, 1], [1, 0]]
a = counts_to_freqs([df.iloc[0,5],df.iloc[0,6]])
b = counts_to_freqs([df.iloc[1,5],df.iloc[1,6]])
#eps = 200
#print(type(df.iloc[1][1]))
#simple ot test
ot.emd2(a, b, M)


0.45316839065367426

In [112]:
file_name = 'phen.xlsx'

df = pd.read_excel(io = file_name, header = [0, 1])
print(df.head(5))  # print first 5 rows of the dataframe

  Phenylalanine (Phe/F)                  ...                                    
                  # CDS Organelle TaxID  ...     # TTC TTT-Fraction TTC-Fraction
0               3888949   genomic   562  ...  18665561    57.772840    42.227160
1               2710660   genomic   520  ...  24457252    12.456001    87.543999
2               2141118   genomic   573  ...  12262306    49.495208    50.504792
3               1387878   genomic   287  ...  14576929     6.217090    93.782910
4               1462644   genomic  1280  ...   4995245    73.219726    26.780274

[5 rows x 9 columns]


In [113]:
#just a test line: we want to know that all of them are 'genomic'
#and, we look hows to refer to columns
all(df.iloc[:,1]=='genomic')

True

In [114]:
#we remove short things that are phages, etc -- here it makes sense to plot
#the distribution of all the stuff by length, probaly it is bimodal
df=df[df.iloc[:,3]>=100000]

In [115]:
orgainm_name=df.iloc[:,4]
esch_index=orgainm_name.str.contains('Escherichia',regex=False)
kleb_index=orgainm_name.str.contains('Klebsiella',regex=False)
df[esch_index | kleb_index]


Phenylalanine (Phe/F)            ...                          
                     # CDS Organelle  ... TTT-Fraction TTC-Fraction
0                  3888949   genomic  ...    57.772840    42.227160
2                  2141118   genomic  ...    49.495208    50.504792
25                  231131   genomic  ...    49.425862    50.574138
44                  168232   genomic  ...    48.828644    51.171356
57                  128713   genomic  ...    58.385716    41.614284
...                    ...       ...  ...          ...          ...
8924                   588   genomic  ...    66.304120    33.695880
8929                   576   genomic  ...    57.554567    42.445433
8935                   534   genomic  ...    69.411033    30.588967
8943                   542   genomic  ...    74.259780    25.740220
8958                   551   genomic  ...    66.051260    33.948740

[198 rows x 9 columns]

In [116]:
#extract type of bacteria
#import xlrd
#import xlsxwriter

#wb_kleb = xlsxwriter.Workbook('Kleb.xlsx')
#wk = wb_kleb.add_worksheet()
#count_kleb = 0

#wb_ecoli = xlsxwriter.Workbook('Ecoli.xlsx')
#we = wb_ecoli.add_worksheet()
#count_ecoli = 0


workbook = xlrd.open_workbook('phen.xlsx')
for sheet in workbook.sheets():
    for row in range(sheet.nrows):
        #print(row)
        if (row > 1):
            q = sheet.row(row)[4].value.split()
            q = set(q)
            if (len(q.intersection({'Klebsiella'})) > 0):
                count_kleb = count_kleb + 1
                l = sheet.row(row)
                for n in range(len(l)):
                    wk.write(count_kleb, n, l[n].value)
               
            elif (len(q.intersection({'Escherichia'})) > 0):
                count_ecoli = count_ecoli + 1
                l = sheet.row(row)
                for n in range(len(l)):
                    we.write(count_ecoli, n, l[n].value)
wb_kleb.close() 
wb_ecoli.close() 

NameError: ignored

In [ ]:
#read data
ecoli = []
dfe = pd.read_excel(io = 'Ecoli.xlsx')
for n in range(len(dfe)):
    ecoli.append(extract_freq_new(dfe.iloc[n, 5], dfe.iloc[n, 6]))
    
kleb = []
dfk = pd.read_excel(io = 'Kleb.xlsx')
for n in range(len(dfk)):
    kleb.append(extract_freq_new(dfk.iloc[n, 5], dfk.iloc[n, 6]))

out = ecoli + kleb   

wb_dist = xlsxwriter.Workbook('dist1.xlsx')
w = wb_dist.add_worksheet()

for n in range(len(out)):
    for m in range(len(out)):
        if (m <= n):
            w.write(n, m, ot.emd2(out[n], out[m], M))


wb_dist.close()  



In [ ]:
import matplotlib.pyplot as plt
dist = pd.read_excel(io = 'dist1.xlsx', header = [0])


In [ ]:
print(len(ecoli), len(kleb))
print(np.mean(dist[0][0:751]))
print(np.mean(dist[0][752:len(out)]))

In [ ]:
print(m,n,len(out))

In [ ]:
print(type(dist))

In [ ]:
print(np.mean(dist.iloc[1:751, 0]))
print(np.mean(dist.iloc[752:1102, 0]))

#print(np.mean(dist.iloc[len(out), 1:751]))
#print(np.mean(dist.iloc[len(out), 752:len(out)]))
print(np.mean(dist.iloc[1101, 0:751]))
print(np.mean(dist.iloc[1101, 752:1101]))

In [ ]:
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt



# Prepare a vector of color mapped to the 'cyl' column
my_palette = dict(zip(df.cyl.unique(), ["orange","yellow","brown"]))
row_colors = df.cyl.map(my_palette)
 
# plot
sns.clustermap(df, metric="correlation", method="single", cmap="Blues", standard_scale=1, row_colors=row_colors)



In [ ]:
df + np.transpose(df)